In [1]:
import os

data from: https://cds.climate.copernicus.eu/cdsapp#!/dataset/insitu-gridded-observations-global-and-regional?tab=doc

https://crudata.uea.ac.uk/cru/data/hrg/index.htm#current

https://crudata.uea.ac.uk/cru/data/hrg/cru_ts_4.07/

https://www.nature.com/articles/s41597-020-0453-3

CRU has a days wet per month variable! that's perfect, even better than precipitation

WET
also has
DTR
which is diurnal temperature range! I need to get that one too

Note 5 in the nature paper confirms it! The min and max temps are means of the daily min max!

In [2]:
from scipy.io import netcdf
import xarray as xr
import numpy as np

In [27]:
class TravelMap:

    def __init__(self, data_dir='/home/shrey/Dev/stats_curriculum/geospatial/data/'):

        self.massage_data(data_dir)
        self.queried_dfs = []

    def massage_data(self, data_dir):
        
        datafiles = os.listdir(data_dir)
        df_list = []
        for file in datafiles:
            ds = xr.open_dataset(data_dir+file)
            df = ds.to_dataframe().reset_index().dropna()
            df_list.append(df)
        df_list = [df.drop(['stn'], axis=1) for df in df_list] # don't need the 'st' column
        df_t= df_list[0]
        for df in df_list[1:]:
            df_t = df_t.merge(df, on=['lon','lat','time'])

        def cels_to_fahr(x):
            """Americuhhh fuck yeahhh"""
            y = 32 + (x*1.8 )
            return y
        
        df_t['tmx_f'] = df_t['tmx'].apply(lambda x: cels_to_fahr(x))
        df_t['tmn_f'] = df_t['tmn'].apply(lambda x: cels_to_fahr(x))
        df_t['tmp_f'] = df_t['tmp'].apply(lambda x: cels_to_fahr(x))

        self.global_df = df_t

    def query_by_lat_lon(self, my_lat = 37.77, my_lon = -122.44):
        """
        Queries the global dataframe by lat and lon and then stories the query as well as the results
        """
        self.queried_df = self.global_df.loc[(np.abs(self.global_df.lon-my_lon)<=0.25) & (np.abs(self.global_df.lat - my_lat)<=0.25)].groupby('time').agg('mean')
        results_dict = {'lat':my_lat, 'lon':my_lon, 'df':self.queried_df}
        self.queried_dfs.append(results_dict)

In [28]:
my_map = TravelMap()
my_map.query_by_lat_lon()

In [31]:
my_map.query_by_lat_lon(my_lat = 40.7, my_lon = -73.9)

In [30]:
my_map.queried_dfs[0]['df']

,lon,lat,pre,tmn,tmx,dtr,wet,tmp,tmx_f,tmn_f,tmp_f
time,,,,,,,,,,,
2021-01-16,-122.25,37.75,116.400002,7.200000,15.900001,8.700000,10 days 07:55:11.993408207,11.500000,60.620001,44.960001,52.700000
2021-02-15,-122.25,37.75,23.100000,7.900000,17.100000,9.200000,5 days 05:31:12.001647950,12.500000,62.780001,46.220000,54.500000
2021-03-16,-122.25,37.75,40.100002,7.200000,17.000000,9.800000,7 days 18:00:00,12.100000,62.600000,44.960001,53.780001
2021-04-16,-122.25,37.75,2.300000,9.300000,20.500000,11.200000,1 days 03:50:23.997116088,14.900001,68.900000,48.740000,58.820001
2021-05-16,-122.25,37.75,0.200000,11.500000,21.700001,10.200000,0 days 06:00:00,16.600000,71.060001,52.700000,61.880001
2021-06-16,-122.25,37.75,4.900000,14.300000,24.600000,10.300000,0 days 23:31:11.996498104,19.400000,76.280001,57.740000,66.919999
2021-07-16,-122.25,37.75,0.500000,14.700000,25.000000,10.300000,0 days 08:52:48.000411989,19.800001,77.000000,58.460000,67.640002
2021-08-16,-122.25,37.75,0.000000,14.400001,24.100000,9.700000,0 days 00:14:23.999980689,19.200001,75.380001,57.920001,66.560001
2021-09-16,-122.25,37.75,0.900000,14.500000,25.000000,10.500000,0 days 12:57:35.996704099,19.700001,77.000000,58.100000,67.460001


In [32]:
my_map.queried_dfs[1]['df']

,lon,lat,pre,tmn,tmx,dtr,wet,tmp,tmx_f,tmn_f,tmp_f
time,,,,,,,,,,,
2021-01-16,-73.75,40.75,55.200001,-1.700000,4.900000,6.600000,9 days 09:21:35.947265621,1.600000,40.820000,28.940000,34.880000
2021-02-15,-73.75,40.75,112.700005,-2.500000,3.900000,6.400000,12 days 18:57:35.996704099,0.700000,39.020000,27.500000,33.260000
2021-03-16,-73.75,40.75,98.900002,1.800000,12.100000,10.300000,7 days 09:21:35.988464355,6.900000,53.780001,35.240000,44.420000
2021-04-16,-73.75,40.75,105.200005,6.600000,17.000000,10.400001,10 days 06:28:47.957153324,11.800000,62.600000,43.880000,53.240000
2021-05-16,-73.75,40.75,118.700005,11.300000,22.000000,10.700000,11 days 14:52:47.990112306,16.600000,71.600000,52.340000,61.880001
2021-06-16,-73.75,40.75,75.099998,17.900000,28.000000,10.100000,10 days 17:16:47.940673828,22.900000,82.400000,64.219999,73.219999
2021-07-16,-73.75,40.75,215.699997,20.400000,29.500000,9.100000,14 days 02:52:47.990112306,24.900000,85.100000,68.719999,76.819999
2021-08-16,-73.75,40.75,166.600006,21.300001,29.800001,8.500000,12 days 14:09:36.013183594,25.500000,85.640002,70.340002,77.900000
2021-09-16,-73.75,40.75,157.800003,16.900000,25.900000,9.000000,11 days 01:12:00.016479495,21.400000,78.619999,62.419999,70.519999
